In [1]:
"""
Import necessary libraries
"""
import pandas as pd
import numpy as np
import matplotlib_inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

In [2]:
"""
Read in the data
"""
opportunities = pd.read_excel('Opportunity.xlsx')
NAICS = pd.read_excel('NAICS.xlsx')

/Users/benkarabinus/.local/share/virtualenvs/FinalProjectLocal-uhAi4IsK/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


# Exploratory Data Analysis

## *Data Cleaning and Feature Engineering*

### Objectives
- Column naming convention
- NAICS categories must be joined with opportunities
- System close date should be subtracted from record creation date to creaate a new feature representing days an opportunity is active 
- Remove non-informative data
- Handle missing data
- Separate categorical and continuous features for further analysis

In [3]:
opportunities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3894 entries, 0 to 3893
Columns: 114 entries, (Do Not Modify) Opportunity to PFY RSM Services Spend (Account) (Account)
dtypes: datetime64[ns](12), float64(29), int64(1), object(72)
memory usage: 3.4+ MB


In [4]:
opportunities.isnull().sum()[0:26]

(Do Not Modify) Opportunity                          0
(Do Not Modify) Row Checksum                         0
(Do Not Modify) Modified On                          0
Opportunity Name                                     0
Account                                              0
Entity ID (Account) (Account)                       31
Address 1: Street 1 (Account) (Account)             11
Address 1: City (Account) (Account)                 10
Address 1: State/Province (Account) (Account)       14
Address 1: County (Account) (Account)              218
Address 1: Country/Region (Account) (Account)       10
Address 1: ZIP/Postal Code (Account) (Account)      13
Address 1: Latitude (Account) (Account)            148
Address 1: Longitude (Account) (Account)           148
Contact                                            173
Description                                       3360
LOB/Service Line                                     0
Solution Set                                         0
Currency  

In [5]:
opportunities.isnull().sum()[26:51]

Account Rating (Account) (Account)                   1718
MSA (Account) (Account)                               151
No. of Employees (Account) (Account)                   65
Original Opportunity                                 3496
Open Deals (Account) (Account)                          0
Open Revenue (Account) (Account)                        0
Revenue This Year Aggregated (Account) (Account)     1429
Type of Government Contractor (Account) (Account)    3651
Customer Need                                        3352
Engagement Type                                      3605
Rating                                                  0
Owner                                                   0
Account Mgmt 1 (Account) (Account)                   1814
Account Mgmt 2 (Account) (Account)                   3865
Relationship Lead (Account) (Account)                   0
Consulting Lead (Account) (Account)                  1019
Region (Account) (Account)                             96
Annual Revenue

In [6]:
opportunities.isnull().sum()[51:76]

Actual Close Date                                0
System Close Date                                0
Status Reason                                    0
Sales Stage                                      0
Status                                           0
Collaboration-Approved Date                   3825
Referral Contact                              3882
Approval Status                               3894
Commissonable                                  895
Created By                                       0
PSE 1                                         3527
PSE 2                                         3772
CJP Target (Account) (Account)                1786
CFY RSM Services Spend (Account) (Account)     896
Current Situation                             3894
D&B Company (Account) (Account)                555
Deliverability Rating (Contact) (Contact)     1483
D&B Financial (Account) (Account)             3894
EAL (Account) (Account)                       3508
Import Date (Account) (Account)

In [7]:
opportunities.isnull().sum()[76:101]

Microsoft Enterprise Agreement (Account) (Account)        1446
Microsoft Reference Program (Account) (Account)           2887
NAICS (Account) (Account)                                  443
National Initiative (Account) (Account)                   1985
Other Software (Account) (Account)                        3840
Preferred Method of Contact (Account) (Account)              0
Preferred Service (Account) (Account)                     3894
Private Equity (Account) (Account)                         459
Referral Notes (Account) (Account)                        3842
Referring Firm (Account) (Account)                        3894
Relationship Strength (IH) (Account) (Account)            1286
Revenue Last Year (Account) (Account)                     3894
Revenue Last Year (Base) (Account) (Account)              3894
Revenue This Year (Account) (Account)                     3894
Revenue This Year (Base) (Account) (Account)              3894
Territory (Account) (Account)                          

In [8]:
opportunities.isnull().sum()[101:120]

Business Type (Potential Customer) (Account)                    170
Category (Potential Customer) (Account)                        3894
Client Status (Potential Customer) (Account)                    939
D&B Company (Potential Customer) (Account)                      700
D&B Financial (Potential Customer) (Account)                   3894
D&B Match Flag (Potential Customer) (Account)                  2800
D&B NAICS (Potential Customer) (Account)                        213
D&B Ultimate Parent (Potential Customer) (Account)             3894
Market Capitalization (Potential Customer) (Account)           3894
Market Capitalization (Base) (Potential Customer) (Account)    3894
National Initiative (Potential Customer) (Account)             2018
Other Software (Potential Customer) (Account)                  3844
PFY RSM Services Spend (Account) (Account)                      896
dtype: int64

*Rename columns for ease of use and readability*

In [9]:
opportunities.columns = opportunities.columns.str.replace('\([^)]*\)', '')
opportunities.columns = opportunities.columns.str.replace(':.', '')
opportunities.columns = opportunities.columns.str.replace('/', '_')
opportunities = opportunities.rename(columns=lambda x : x.strip())
opportunities.columns = opportunities.columns.str.replace(' ','_')
opportunities.columns = opportunities.columns.str.replace('Address_1', '')

/var/folders/s4/yckpqxmx6cnf837zqjqn_hxc0000gn/T/ipykernel_43805/430631756.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  opportunities.columns = opportunities.columns.str.replace('\([^)]*\)', '')
/var/folders/s4/yckpqxmx6cnf837zqjqn_hxc0000gn/T/ipykernel_43805/430631756.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  opportunities.columns = opportunities.columns.str.replace(':.', '')


In [10]:
# rename system systen generated GUID to opportunityID
opportunities.rename(columns={opportunities.columns[0]: 'opportunity_id'}, inplace=True)
print(opportunities.columns[0])

opportunity_id


In [11]:
"""
Sanity check
"""
for column in list(opportunities.columns):
    print(column)

opportunity_id
Row_Checksum
Modified_On
Opportunity_Name
Account
Entity_ID
Street_1
City
State_Province
County
Country_Region
ZIP_Postal_Code
Latitude
Longitude
Contact
Description
LOB_Service_Line
Solution_Set
Currency
Est._Fees
Est._Revenue
Total_Amount
Lead_Source
Type
Probability
AccountOriginal_Source
Account_Rating
MSA
No._of_Employees
Original_Opportunity
Open_Deals
Open_Revenue
Revenue_This_Year_Aggregated
Type_of_Government_Contractor
Customer_Need
Engagement_Type
Rating
Owner
Account_Mgmt_1
Account_Mgmt_2
Relationship_Lead
Consulting_Lead
Region
Annual_Revenue
Business_Developer_1
Business_Developer_2
Import_Date
Record_Created_On
Created_On
Modified_On
Est._Close_Date
Actual_Close_Date
System_Close_Date
Status_Reason
Sales_Stage
Status
Collaboration-Approved_Date
Referral_Contact
Approval_Status
Commissonable
Created_By
PSE_1
PSE_2
CJP_Target
CFY_RSM_Services_Spend
Current_Situation
D&B_Company
Deliverability_Rating
D&B_Financial
EAL
Import_Date
Internationally_Active
IPM_Da

In [12]:
opportunities['NAICS'] = opportunities['NAICS'].str.replace(r'\D', '')
opportunities['NAICS'] = opportunities['NAICS'].str.strip()

/var/folders/s4/yckpqxmx6cnf837zqjqn_hxc0000gn/T/ipykernel_43805/389924680.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  opportunities['NAICS'] = opportunities['NAICS'].str.replace(r'\D', '')


In [13]:
opportunities['NAICS'] = opportunities['NAICS'].fillna(value=0)
opportunities['NAICS'].isnull().sum()
opportunities['NAICS'] = opportunities['NAICS'].astype(int)

In [14]:
opportunities = pd.merge(opportunities, NAICS, how='inner', left_on = ['NAICS'],  right_on = ['mcg_naicscode'])

*Handle Missing and Non-informative Data*

- Drop columns containing non-informative data
- Drop all features missing 50% or more data
- Replace missing values for categorical features with "\_missing_"
- Replace missing numerical features with appropriate values or drop if necessary

In [15]:
opportunities.shape

(3451, 121)

In [16]:
# drop unecessary columns
opportunities.drop(columns=['Row_Checksum', 'LOB_Service_Line', 'Solution_Set', 'D&B_Company', 
                            'D&B_Financial', 'IPM_Date_Acquired', 'IPM_Date_Lost', 'Import_Date', 
                            'Record_Created_On', 'Modified_On', 'Actual_Close_Date', 'Sales_Stage', 
                            'Original_Opportunity', 'D&B_Company', 'Assets', 'Opportunity_Name', 'Account', 
                            'Est._Fees', 'Total_Amount', 'Revenue_This_Year_Aggregated', 'Status_Reason', 'MSA',
                            'Rating', 'Consulting_Lead', 'Region', 'Deliverability_Rating', 'Relationship_Strength', 'mcg_naicscode',
                            'mcg_name', 'mcg_sicdescription', 'Commissonable', 'CJP_Target', 'AccountOriginal_Source', 'Business_Type',
                            'D&B_NAICS', 'National_Initiative', 'Territory_Code', 'Last_Interaction', 'NAICS'],
                            axis=1, inplace=True)

print("Dropping non-informative features reduces the dimension of the dataset to {} features.".format(len(opportunities.columns)))

Dropping non-informative features reduces the dimension of the dataset to 77 features.


In [17]:
for i in range(len(opportunities.columns)):
    print(i, opportunities.columns[i])

0 opportunity_id
1 Entity_ID
2 Street_1
3 City
4 State_Province
5 County
6 Country_Region
7 ZIP_Postal_Code
8 Latitude
9 Longitude
10 Contact
11 Description
12 Currency
13 Est._Revenue
14 Lead_Source
15 Type
16 Probability
17 Account_Rating
18 No._of_Employees
19 Open_Deals
20 Open_Revenue
21 Type_of_Government_Contractor
22 Customer_Need
23 Engagement_Type
24 Owner
25 Account_Mgmt_1
26 Account_Mgmt_2
27 Relationship_Lead
28 Annual_Revenue
29 Business_Developer_1
30 Business_Developer_2
31 Created_On
32 Est._Close_Date
33 System_Close_Date
34 Status
35 Collaboration-Approved_Date
36 Referral_Contact
37 Approval_Status
38 Created_By
39 PSE_1
40 PSE_2
41 CFY_RSM_Services_Spend
42 Current_Situation
43 EAL
44 Internationally_Active
45 IPM_Lost_Reason
46 Microsoft_Enterprise_Agreement
47 Microsoft_Reference_Program
48 Other_Software
49 Preferred_Method_of_Contact
50 Preferred_Service
51 Private_Equity
52 Referral_Notes
53 Referring_Firm
54 Revenue_Last_Year
55 Revenue_Last_Year
56 Revenue_T

In [18]:
opportunities.rename(columns={'mcg_industry_displayname' : 'industry','mcg_industrysector_displayname' : 'industry_sector', 
                                'mcg_subsectortext0' : 'industry_subsector', 'mcg_description' : 'description'}, inplace=True)

In [19]:
print(opportunities['Created_On'].dtype)
print(opportunities['Est._Close_Date'].dtype)
print(opportunities['System_Close_Date'].dtype)

datetime64[ns]
datetime64[ns]
datetime64[ns]


In [20]:
opportunities['days_active'] = opportunities['System_Close_Date'] - opportunities['Created_On']
opportunities['days_active'] = opportunities['days_active'] / np.timedelta64(1, 'D')
opportunities['days_active'][opportunities['days_active'] < 0].count()

858

In [21]:
problem_dates = \
opportunities[['days_active', 'Created_On', 'System_Close_Date']][opportunities['days_active'] < 0]

Creating the column "days_active" and counting days_active values < 0 shows a total of 858 opportunities that were active for a negative amount of days. This is likely due to errors during data entry or import of records into the customer relationship management system.

In [22]:
opportunities[['opportunity_id', 'Entity_ID']].isna().sum()

opportunity_id    0
Entity_ID         0
dtype: int64

In [23]:
# drop columns missing more than 50% of values
perc = 0.50
threshold = int(perc*opportunities.shape[0])
opportunities.dropna(axis=1, thresh=threshold, inplace=True)
print("Dropping features missing 50 percent or more data reduces the dimension of the dataset"
" to {} features.".format(len(opportunities.columns)))

Dropping features missing 50 percent or more data reduces the dimension of the dataset to 42 features.


Separate continuous and categorical features

In [24]:
# seperate numerical and categorical columns
opportunityID = opportunities[['opportunity_id', 'Entity_ID']].copy()
opp_numeric = opportunities.select_dtypes(include=[np.number])
opp_non_numeric = opportunities.select_dtypes(exclude=[np.number])
opp_non_numeric.drop(columns=['City','Street_1', 'State_Province', 'Country_Region','County',
                             'ZIP_Postal_Code'], inplace=True)
opp_geographic = opportunities.iloc[:, 0:10].copy()

In [25]:
opp_numeric = pd.merge(opportunityID, opp_numeric, left_index=True, right_index=True, how='inner')
opp_non_numeric = pd.merge(opportunityID, opp_non_numeric, left_index=True, right_index=True, how='inner')
opp_numeric.drop(columns=['Entity_ID_y', 'Latitude', 'Longitude'], axis=1, inplace=True)
opp_non_numeric.drop(columns=['opportunity_id_y'], axis=1, inplace=True)
opp_numeric.rename(columns={'Entity_ID_x' : 'Entity_ID'}, inplace=True)
opp_non_numeric.rename(columns={'opportunity_id_x' : 'opportunity_id'}, inplace=True)
opp_numeric.set_index(['opportunity_id', 'Entity_ID'], inplace=True)
opp_non_numeric.set_index(['opportunity_id', 'Entity_ID'], inplace=True)
opp_geographic.set_index(['opportunity_id', 'Entity_ID'], inplace=True)

In [26]:
# sanity check
print(opp_numeric.shape)
print(opp_non_numeric.shape)

(3451, 8)
(3451, 24)


In [27]:
opp_non_numeric.isnull().sum()

Contact                             94
Currency                             0
Lead_Source                          3
Type                                 0
Probability                        219
Account_Rating                    1501
Owner                                0
Account_Mgmt_1                    1385
Relationship_Lead                    0
Created_On                           0
Est._Close_Date                      0
System_Close_Date                    0
Status                               0
Created_By                           0
Internationally_Active            1181
Microsoft_Enterprise_Agreement    1245
Preferred_Method_of_Contact          0
Private_Equity                     375
VIP_Client?                          0
Client_Status                      497
industry_sector                      1
industry                             1
industry_subsector                   1
description                          0
dtype: int64

In [28]:
# fillna for categorical features with value "_missing_"
opp_non_numeric.fillna(value='_missing_', inplace=True)
opp_non_numeric.isnull().sum()

Contact                           0
Currency                          0
Lead_Source                       0
Type                              0
Probability                       0
Account_Rating                    0
Owner                             0
Account_Mgmt_1                    0
Relationship_Lead                 0
Created_On                        0
Est._Close_Date                   0
System_Close_Date                 0
Status                            0
Created_By                        0
Internationally_Active            0
Microsoft_Enterprise_Agreement    0
Preferred_Method_of_Contact       0
Private_Equity                    0
VIP_Client?                       0
Client_Status                     0
industry_sector                   0
industry                          0
industry_subsector                0
description                       0
dtype: int64

In [29]:
opp_numeric.isnull().sum()

Est._Revenue                0
No._of_Employees           17
Open_Deals                  0
Open_Revenue                0
Annual_Revenue             18
CFY_RSM_Services_Spend    453
PFY_RSM_Services_Spend    453
days_active                 0
dtype: int64

Printing a count of null values for each continuous variable shows that the columns "CFY_RSM_Services_Spend" and "PFY_RSM_Services_Spend" are both missing 453 observations each. It's assumed that missing values in these columns represent clients who have not purchased consulting services from the firm in either of the two periods. As such, missing values will be replaced with 0. Furthermore, each of the remaining continuous predictors missing data represent a small fraction of the total record count and should be dropped to improve model performance.

In [30]:
# fill current fiscal year spend and previous fiscal year spend with 0, assuming if value not given no prior or current spend
opp_numeric['CFY_RSM_Services_Spend'] = opp_numeric['CFY_RSM_Services_Spend'].fillna(value=0)
opp_numeric['PFY_RSM_Services_Spend'] = opp_numeric['PFY_RSM_Services_Spend'].fillna(value=0)

In [31]:
# Drop rows where annual revenue or number of employees is missing
opp_numeric = opp_numeric.dropna(subset=['Annual_Revenue', 'No._of_Employees'])

In [32]:
# print dimensions
opp_numeric.shape

(3433, 8)

In [33]:
# sanity check
opp_numeric.isnull().sum()

Est._Revenue              0
No._of_Employees          0
Open_Deals                0
Open_Revenue              0
Annual_Revenue            0
CFY_RSM_Services_Spend    0
PFY_RSM_Services_Spend    0
days_active               0
dtype: int64

## *Explore Categorical Features*

Objectives
- 

In [35]:
opp_non_numeric['Lead_Source'].value_counts()

Inactive - Junction Solutions         961
Partner                               492
BD Activity                           473
Director                              383
Referral - Other                      366
Inside Sales                          164
Manager                               153
Referral - Microsoft                   91
Marketing Campaign - National          87
Website                                77
National Call Center                   58
Reputation                             41
Other                                  40
RSM System Selection                   38
RFP                                     8
COI                                     4
_missing_                               3
Marketing Campaign - Regional           3
RSM Alumni                              2
Senior Director                         2
RSM Alliance                            2
Trade Show                              1
Inactive - Explore Consulting, LLC      1
RSM Int'l Inbound                 

In [36]:
opp_non_numeric.value_counts('Type')

Type
Existing Relationship - New Business    1741
Existing Relationship - Renewal          905
New Relationship - New Business          805
dtype: int64

In [37]:
opp_non_numeric.value_counts('Owner')

Owner
Chris Hafenscher    326
Michael Webster     264
Debby Smith         256
Jay Jordan          201
Seth Derevensky     158
                   ... 
Paul Vial             1
Paul Calamita         1
John Harder           1
John Jones            1
Kerensa Butler        1
Length: 236, dtype: int64

In [ ]:
## *Explore Continuous Variables*

### Methods

- Print Summary Statistics
- Distribution Plots

In [ ]:
# print summary statistics
pd.set_option('display.float_format', lambda x: '%.4f' % x)
opp_numeric.iloc[:, 3:].describe()

## *Explore Relationship Between Variables*

### Methods

- Pearson's Correlation

In [ ]:
opp_numeric.iloc[:, 3:].corr()

In [ ]:
# plot correlation of numeric data
sns.set(rc={"figure.figsize":(10, 6)})
correlation = sns.heatmap(opp_numeric.iloc[:, 3:].corr(), 
                          annot=True).set_title('Pearson\'s Correlation: Opportunity Features')

In [ ]:
fig = px.histogram(opp_numeric, x='Est._Revenue')
fig.show()

In [ ]:
fig = px.box(opp_numeric, y='No._of_Employees')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=x0))
fig.add_trace(go.Histogram(x=x1))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
opportunities = pd.merge(opp_numeric, opp_non_numeric, left_index=True, right_index=True, how='inner')
#opportunities = opp_numeric.join(opp_non_numeric, lsuffix='left', rsuffix='right')
for i in range(len(opportunities.columns)):
    print(i, opportunities.columns[i])

In [ ]:
opp_non_numeric.shape

In [ ]:
opportunities.isnull().sum()

In [ ]:
opportunities.to_excel('Opportunity_reduced.xlsx')